# 1. Imports

In [11]:
import sys

In [12]:
sys.version_info

sys.version_info(major=3, minor=10, micro=15, releaselevel='final', serial=0)

In [13]:
assert sys.version_info >= (3, 6), "Python 3.6 or later is required"

In [19]:
from packaging import version
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [15]:
sklearn.__version__

'1.6.0'

In [16]:
assert version.parse(sklearn.__version__) >= version.parse("1.0.1"), "scikit-learn 1.0.1 or later is required"

In [17]:
plt.rc('font', size=12)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=12)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [18]:
np.random.seed(88)